In [1]:
!pwd

/content


In [0]:
!pip install -U tensorflow==1.15
!pip install object_detection

Requirement already up-to-date: tensorflow==1.15 in /usr/local/lib/python3.6/dist-packages (1.15.0)


In [2]:
!pip install pycocotools

In [3]:
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3380, done.
remote: Counting objects: 100% (3380/3380), done.
remote: Compressing objects: 100% (2871/2871), done.
remote: Total 3380 (delta 653), reused 2048 (delta 431), pack-reused 0
Receiving objects: 100% (3380/3380), 371.08 MiB | 36.17 MiB/s, done.
Resolving deltas: 100% (653/653), done.
Checking out files: 100% (3194/3194), done.


In [0]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

In [6]:
%%bash 
cd models/research
pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp36-none-any.whl size=1017523 sha256=272d796aa89f12c771e1e334a7984c506f71ae0a6ccb32800a926dc4d7b07eaa
  Stored in directory: /tmp/pip-ephem-wheel-cache-dv9u7n1u/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
Successfully built object-detection


In [7]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

In [0]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [0]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

In [11]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [12]:
!pwd

/content


In [0]:
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [14]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [15]:
%cd '/content/gdrive/My Drive/object_detection'

/content/gdrive/My Drive/object_detection


In [16]:
!python transform_image_resolution.py -d images/ -s 800 600

Successfully converted xml to csv.


In [17]:
!python generate_tfrecord.py --csv_input=images/train_labels.csv --image_dir=images/train --output_path=train.record


W0120 19:00:04.128848 140636785952640 module_wrapper.py:139] From generate_tfrecord.py:86: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.


W0120 19:00:04.590963 140636785952640 module_wrapper.py:139] From generate_tfrecord.py:45: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

Successfully created the TFRecords: /content/gdrive/My Drive/object_detection/train.record


In [18]:
!python generate_tfrecord.py --csv_input=images/test_labels.csv --image_dir=images/test --output_path=test.record


W0120 19:00:17.432859 139907315324800 module_wrapper.py:139] From generate_tfrecord.py:86: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.


W0120 19:00:17.907136 139907315324800 module_wrapper.py:139] From generate_tfrecord.py:45: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

Successfully created the TFRecords: /content/gdrive/My Drive/object_detection/test.record


UsageError: Line magic function `%tensorboard` not found.


In [19]:
!python model_main.py --logtostderr --model_dir=training/ --pipeline_config_path=training/faster_rcnn_inception_v2_pets.config

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





W0120 19:00:40.590808 140328429365120 module_wrapper.py:139] From /usr/local/lib/python3.6/dist-packages/object_detection/utils/config_util.py:102: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0120 19:00:40.839941 140328429365120 module_wrapper.py:139] From /usr/local/lib/python3.6/dist-packages/object_detection/model_lib.py:628: The name tf.logging.warning is deprecated. Please use tf.compat.v1.logging.warning instead.

W0120 19:00:40.840173 140328429365120 model_lib.py:629] Forced number of epochs for all eval validations to be 1.

W0120 19:00:40.840334 140328429365120 module_wrapper.py:1

KeyboardInterrupt: ignored

In [5]:
!python export_inference_graph.py --input_type image_tensor --pipeline_config_path training/faster_rcnn_inception_v2_pets.config --trained_checkpoint_prefix training/model.ckpt-618 --output_directory inference_graph

python3: can't open file 'export_inference_graph.py': [Errno 2] No such file or directory
